<a href="https://colab.research.google.com/github/artms-18/tensorflow_fundamentals/blob/main/Intro_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in Tensorflow

NLP has the goal of deriving information out of natural language (could be sequences text or speech).

Another common term for NLP problems is sequence to sequence problems (seq2seq)

## Check for GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-81ae9ec0-46b1-3ebe-5bdd-4d33dafa5eb8)


## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import a series of helper function for the notebook

from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2021-05-25 12:08:23--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-05-25 12:08:23 (76.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataaset

The dataset we're going to be using is Kaggle's introduction to NLP dataset

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

unzip_data("nlp_getting_started.zip")

--2021-05-25 12:08:25--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 172.253.117.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2021-05-25 12:08:26 (104 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualie our text samples, we first have to read them in, one way to do so would be to use Python: 

Another way to do this is to use pandas.

(If text dataset is very large, cannot ise pandas (will tkaes of to much RAM).



In [4]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv('test.csv')

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [6]:
# Shuffle training dataframe

train_df_shuffled = train_df.sample(frac = 1, random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
# What does the test dataframe look like?

test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [10]:
# Let's visualize some random training examples

import random

random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples

for row in train_df_shuffled[['text', 'target']][random_index: random_index + 5].itertuples():
  _, text, target = row #gets rid of index
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster
Text:
Dead Space - Obliteration Imminent [2/2]: http://t.co/XJB0dCAaHf via @YouTube

---

Target: 0 (not real disaster
Text:
She just said does he have a curfew 'nope'??

---

Target: 1 (real disaster)
Text:
Dr. Bengston on #wildfire management: ÛÏnumbers and size of fires areas affected and costs of fighting them all show upward trend.Û #smem

---

Target: 1 (real disaster)
Text:
#Glimpses: Hyderabad deluged by heavy rainfall | http://t.co/DctV1uJLHc http://t.co/QOx1jNQSAU

---

Target: 0 (not real disaster
Text:
Worked at a fast food joint. Poured burnt hot oil down the sink. It hit the water in the trap and exploded.  @FallonTonight #WORSTSUMMERJOB

---



### Split data into training and validation sets

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
# Use train_test_split top split training data into training and validation sets

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(), train_df_shuffled['target'].to_numpy(),
                                                                            test_size = 0.1, random_state = 42)

In [13]:
# Check the lengths

len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [14]:
# Check the first 10 samples

train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a midek is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenization - direct mapping of  token( a toke could be a word or a character) to number
* embedding - create a matrix of feature vectors for each token the size of the feature vector can be defined and this embedding can be learned

### Text vectorization (tokenization)

In [15]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [16]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters

text_vectorizer = TextVectorization(max_tokens = None, # how many words in the vocab (automatically add <OOV>)
                                    standardize = "lower_and_strip_punctuation",
                                    split = "whitespace", 
                                    ngrams = None,
                                    output_mode = "int", #how to map tokens to numbers
                                    output_sequence_length = None, #pads each sequence to the logest sequence
                                    pad_to_max_tokens = True)

In [17]:
# Find the average number of tokens (words) in the training tweets

round(sum(len(i.split()) for i in train_sentences)) / len(train_sentences)

14.901036345059115

In [18]:
# Setup text vectorization variables

max_vocab_length = 10000 #max num of words to have in our vocabulary
max_length = 15 # max length our seuqences will be (how many words in a tweet, amodel would see)

text_vectorizer = TextVectorization(max_tokens = max_vocab_length, output_mode= 'int', output_sequence_length = max_length)

In [19]:
#fit the text vectorier to the training text

text_vectorizer.adapt(train_sentences)

In [20]:
# Create a sample sentence and tokenize it

sample_sentence = "Theres a flood in my street!"

text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [21]:
# Choose a random sentence from the training dataset and tokenize iot

random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 Driver rams car into Israeli soldiers wounds 3: military: A driver rammed a car into a group of Israeli soldi... http://t.co/oBSZ45ybAJ

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 900, 9366,  127,   66,  602, 1362,  523,  118,  223,    3,  900,
        4808,    3,  127,   66]])>

In [22]:
# Get the unique words in the vocabulary

words_in_vocab = text_vectorizer.get_vocabulary() #get all of the unique words in the training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] #get the least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most comon words: {top_5_words}")

Number of words in vocab: 10000
5 most comon words: ['', '[UNK]', 'the', 'a', 'in']


### Creating an Embedding using an Embedding Layer

To make our embedding, we're going to use tensoflows embedding layer

The parameters we actually care about:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vecotr 100 long
* `input_length` = length of the sequences being passed to the embedding layer

In [23]:
from tensorflow.keras import layers

# in keras, wordks best when numbers are divisible by eight

embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 128,
                             input_length = max_length
                             )
embedding

In [24]:
# Get a random sentence from the training set

random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\n\nEmbedded version:")

# Embed the random sentence (turn into dense vecotrs of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 http://t.co/eHKLp12yiP Paci?c Media Centre | articles: AUSTRALIA: RSF protests over new security gag over reporting on...

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.04794755, -0.02298006,  0.04445425, ..., -0.00928714,
         -0.00892459, -0.02337693],
        [ 0.04794755, -0.02298006,  0.04445425, ..., -0.00928714,
         -0.00892459, -0.02337693],
        [-0.00561165, -0.04215204,  0.04899217, ..., -0.02823051,
          0.03362048,  0.02974412],
        ...,
        [-0.03155226,  0.04727731, -0.01535205, ...,  0.00238422,
          0.03769321,  0.02143246],
        [-0.01056764, -0.03080753, -0.04830956, ..., -0.02159681,
          0.02547295,  0.04991187],
        [-0.00074872, -0.03843345,  0.01336229, ..., -0.01739439,
          0.01713463,  0.01674726]]], dtype=float32)>

In [25]:
# Chekc out a single token's embedding

sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.04794755, -0.02298006,  0.04445425,  0.02801175, -0.00493209,
         0.00407755,  0.01126187,  0.02828297, -0.02486614, -0.02081866,
        -0.00411118, -0.04539023,  0.02249254, -0.01828203, -0.02451078,
         0.02437389,  0.04349944, -0.0399128 ,  0.03252829, -0.03368491,
        -0.00245105, -0.00680388,  0.00092493, -0.02370194,  0.03032849,
         0.03583086,  0.02352022,  0.02005031,  0.03918841,  0.04177998,
         0.03083685,  0.04623346,  0.01447998,  0.01569665, -0.04309411,
        -0.00715474,  0.04999209, -0.02541818,  0.04126694, -0.03793336,
         0.03537754, -0.04974531,  0.04501991,  0.00720901, -0.02937691,
        -0.00497725,  0.0348657 ,  0.00640627, -0.00671239,  0.00416658,
         0.01998308,  0.02775519,  0.0279098 ,  0.00886961, -0.04020447,
         0.01167174,  0.0394491 ,  0.02191233,  0.00025148,  0.00870432,
        -0.04516592, -0.03216982, -0.01048793,  0.04765377, -0.01442585,
  

## Modelling a text dataset (running a series of experiments)

Now we're got a wat to turn our text srequences into numbers, it's time to start building a series of modelling experiments, start from a baseline and move on from there 

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neual network (dense model)
* Model 2: GRU model (RNN)
* Model 4: Bidirectional-LSTM
* Model 5: 1d Convolutional Neural Network (CNN)
* Model 6: Tensorflow Hub Pretrained Geature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

How are we going to approach all of these?

Use the standard steps in modelling with tensorflow:

* Create a model
* Build a model
* Compile a model
* Fit the model

### Model 0: Getting a baseline

As with all machine learning modelling experiments it's important to create a baseline model so u got a benchmark for future experiments to build upon

To create our baseline, we'll use SKlearns Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers

> **Note:** It's common practive to use non-DL algorithms as a baseline because of their speed, then later using DL to improve

In [26]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline

model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB())
])

# Fit the pipeline to the training data

model_0.fit(train_sentences, train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [27]:
# Evaluate our baseline model 

baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [28]:
# Make predictions

baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

We could evaluate model's predictin with different methods every time, but this could be cumbersome and could easily be fixed with a function

Let's create one to compare our model's predictions with the truth labels using the following metrics:

* Accuracy
* Precision
* Recall
* F1-score

In [29]:
# Function to evaluate: accuracty, precision, recall, f1-score

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calulates model accuracy, precision, recall and f1 score of a binary classification model.
  """
  #Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  #Calculate model precision, recall ,and f01 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average = 'weighted')
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [30]:
baseline_results = calculate_results(y_true = val_labels, y_pred = baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 1: A simple dense model

In [31]:
# Create a tensorboard callback (need to create a new one for each model)

from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs

SAVE_DIR = "model_logs"

In [32]:
from tensorflow.keras import layers

inputs = layers.Input(shape = (1,), dtype = tf.string) # inputs are 1-dimensional
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numberized imputs
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to one vector
#x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation = "sigmoid")(x) # Create the output layer, want binary output so use sigmoid

model_1 = tf.keras.Model(inputs, outputs, name = "model_1_dense")

In [33]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile mode

model_1.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = ["accuracy"])

In [35]:
# Fit the mode
model_1_history = model_1.fit(x = train_sentences, y = train_labels, epochs = 15, validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR, experiment_name = "model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20210525-120832
Epoch 1/15
215/215 [==============================] - 7s 17ms/step - loss: 0.6536 - accuracy: 0.6428 - val_loss: 0.5379 - val_accuracy: 0.7467
Epoch 2/15
215/215 [==============================] - 3s 14ms/step - loss: 0.4674 - accuracy: 0.8100 - val_loss: 0.4722 - val_accuracy: 0.7848
Epoch 3/15
215/215 [==============================] - 3s 14ms/step - loss: 0.3531 - accuracy: 0.8633 - val_loss: 0.4574 - val_accuracy: 0.7900
Epoch 4/15
215/215 [==============================] - 3s 14ms/step - loss: 0.2857 - accuracy: 0.8937 - val_loss: 0.4647 - val_accuracy: 0.7848
Epoch 5/15
215/215 [==============================] - 3s 13ms/step - loss: 0.2353 - accuracy: 0.9182 - val_loss: 0.4896 - val_accuracy: 0.7900
Epoch 6/15
215/215 [==============================] - 3s 14ms/step - loss: 0.2007 - accuracy: 0.9305 - val_loss: 0.5008 - val_accuracy: 0.7861
Epoch 7/15
215/215 [==============================] - 3s 13ms/step -

In [36]:
# Check the results

model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.7792 - accuracy: 0.7651


[0.7791840434074402, 0.7650918364524841]

In [37]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

(762, 1)

In [38]:
# look at a single prediction

model_1_pred_probs[0] # anything below 0.5 = disaster, abouve = non-=disaster

array([0.44785783], dtype=float32)

In [39]:
# look at the first 10 predictions

model_1_pred_probs[:10]

array([[4.4785783e-01],
       [7.8374606e-01],
       [9.9911612e-01],
       [7.6116130e-02],
       [9.7508985e-04],
       [9.9079883e-01],
       [8.0728340e-01],
       [9.9997318e-01],
       [9.9995100e-01],
       [7.7985579e-01]], dtype=float32)

In [40]:
# Convert model prediction probablilities to label format

model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [41]:
# calculate our model_1 results

model_1_results = calculate_results(y_true = val_labels, y_pred = model_1_preds)

model_1_results

{'accuracy': 76.50918635170603,
 'f1': 0.7631560582067719,
 'precision': 0.7662169502191715,
 'recall': 0.7650918635170604}

In [42]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [43]:
import numpy as np

np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

# Visualizing learned embeddings


In [44]:
# get the vocab from the text vecotrization layer

words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [45]:
# Model 1 summary

model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [46]:
# Get the wieght matrix of embedding layer
# (these are the numerical representations of each token in our training data, which have been learned for 5 epocks)

embed_weights = model_1.get_layer("embedding").get_weights()[0]

print(embed_weights.shape) # same size as vocab size and embedding dim (output dim of our embedding layer)

(10000, 128)


In [47]:
embed_weights

array([[-0.01189457, -0.01891246,  0.03929455, ...,  0.05080683,
         0.03037194, -0.05786855],
       [ 0.04574169, -0.02340131,  0.04712784, ...,  0.00109824,
        -0.00742561, -0.02538231],
       [-0.03239385, -0.0265824 ,  0.00665424, ..., -0.01931636,
         0.03494745, -0.03948031],
       ...,
       [-0.01301078, -0.00147412,  0.03460909, ..., -0.04500416,
         0.0495754 ,  0.02904774],
       [-0.03954068,  0.06667311,  0.04466788, ..., -0.01348996,
         0.02025632, -0.0658415 ],
       [-0.20582347,  0.17602491,  0.16854239, ..., -0.11759412,
         0.15512615, -0.19650836]], dtype=float32)

In [48]:
# Create embedding files

import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

#for index, word in enumerate(words_in_vocab):
  #if index == 0:
   # continue  # skip 0, it's padding.
  #vec = embed_weights[index]
  #out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  #out_m.write(word + "\n")
#out_v.close()
#out_m.close()

In [49]:
# Download files from Colab to upload to projector

#try:
  #from google.colab import files
  #files.download('vectors.tsv')
  #files.download('metadata.tsv')
#except Exception:
  #pass

## Recurrent Neural Networks (RNN's)

RNN's are usefull for sequence data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the respresentation of a later input.

### Model 2: LSTM

LSTM = ling short term memory (one of the most popular LSTM cells)

Our structure of an RNN typically looks like this:

'''
Input (text) -> Tokenize -> EMbedding -> Layers (RNN's/dense) -> Output (label probability)
'''

In [50]:
# Create an LSTM model

from tensorflow.keras import layers

inputs = layers.Input(shape = (1,), dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
#x = layers.LSTM(64, return_sequences = True)(x)
print(x.shape)
x = layers.LSTM(64)(x) # when you're stacking RNN cells together, you need to return_sequences = true
print(x.shape)
#x = layers.Dense(64, activation = 'relu')(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)

model_2 = tf.keras.Model(inputs, outputs, name = "model_2_LSTM")

(None, 15, 128)
(None, 15, 128)
(None, 64)


In [51]:
# Compile the model

model_2.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = ["accuracy"])

In [52]:
model_2_history = model_2.fit(train_sentences, train_labels, epochs = 5, 
                              validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(SAVE_DIR, "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20210525-120921
Epoch 1/5
215/215 [==============================] - 35s 22ms/step - loss: 0.2173 - accuracy: 0.9437 - val_loss: 0.8015 - val_accuracy: 0.7703
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0635 - accuracy: 0.9756 - val_loss: 0.8499 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0626 - accuracy: 0.9756 - val_loss: 1.0573 - val_accuracy: 0.7703
Epoch 4/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0513 - accuracy: 0.9801 - val_loss: 1.0261 - val_accuracy: 0.7664
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0535 - accuracy: 0.9779 - val_loss: 1.0947 - val_accuracy: 0.7664


In [53]:
# Make predictions with LSTM model

model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]


array([[2.9836652e-01],
       [7.9648858e-01],
       [9.9910551e-01],
       [9.3565740e-02],
       [2.5937156e-04],
       [9.9916911e-01],
       [3.0771828e-01],
       [9.9994540e-01],
       [9.9994099e-01],
       [9.9499619e-01]], dtype=float32)

In [54]:
# Convert model 2 pred probs to labels

model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1.], dtype=float32)>

In [55]:
model_2_results = calculate_results(y_true = val_labels, y_pred = model_2_preds)
model_2_results

{'accuracy': 76.64041994750657,
 'f1': 0.7651213533864446,
 'precision': 0.7665895370389821,
 'recall': 0.7664041994750657}

### Model 3: GRU

Another poppular and effective RNN component is the GRU or gated recurrent unit.

The GRU cell has similar features to an LSTM cell but has less parameters.

In [56]:
# Build an RNN using the GRU cell
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
#x = layers.LSTM(64, return_sequences = True)(x)
#print(x.shape)
#x = layers.GRU(64)(x) # make sure return seuqences DOES NO equal true OR add a global average pooling layer instead
#x = layers.Dense(64, activation = "relu")(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)


model_3 = tf.keras.Model(inputs, outputs)

In [57]:
 model_3.summary(
     
 )

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
gru (GRU)                    (None, 64)                37248     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_________________________________________________________________


In [58]:
# COmpile the model

model_3.compile(loss = "binary_crossentropy", optimizer= tf.keras.optimizers.Adam(), metrics = ["accuracy"])


In [59]:
# Fit

model_3_history = model_3.fit(train_sentences,
                              train_labels, epochs = 10,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR, "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20210525-121010
Epoch 1/10
215/215 [==============================] - 6s 20ms/step - loss: 0.2419 - accuracy: 0.8875 - val_loss: 0.9382 - val_accuracy: 0.7651
Epoch 2/10
215/215 [==============================] - 3s 15ms/step - loss: 0.0535 - accuracy: 0.9795 - val_loss: 0.8936 - val_accuracy: 0.7638
Epoch 3/10
215/215 [==============================] - 3s 16ms/step - loss: 0.0485 - accuracy: 0.9784 - val_loss: 0.9261 - val_accuracy: 0.7585
Epoch 4/10
215/215 [==============================] - 3s 16ms/step - loss: 0.0490 - accuracy: 0.9812 - val_loss: 1.0747 - val_accuracy: 0.7559
Epoch 5/10
215/215 [==============================] - 3s 15ms/step - loss: 0.0497 - accuracy: 0.9779 - val_loss: 1.2367 - val_accuracy: 0.7664
Epoch 6/10
215/215 [==============================] - 3s 15ms/step - loss: 0.0430 - accuracy: 0.9806 - val_loss: 1.1372 - val_accuracy: 0.7677
Epoch 7/10
215/215 [==============================] - 3s 15ms/step - l

In [60]:
# Make some predictions with GRu model

model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[1.8600187e-01],
       [7.7952856e-01],
       [9.9996197e-01],
       [1.2717395e-01],
       [2.7786633e-05],
       [9.9987769e-01],
       [9.8702580e-01],
       [9.9998510e-01],
       [9.9997711e-01],
       [9.9413222e-01]], dtype=float32)

In [61]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [62]:
# Calculate model 3 results

model_3_results = calculate_results(y_true = val_labels, y_pred = model_3_preds)
model_3_results

{'accuracy': 76.37795275590551,
 'f1': 0.7629812681164898,
 'precision': 0.7634711836373635,
 'recall': 0.7637795275590551}

### Model 4: Bidirectional RNN

ormal RNN's go from left to right (just like you'd read an English sentence) however, a bidiractional RNN goes from right to left as well as left to right.

In [63]:
# Build a dirirectional RNN in Tensorflow

from tensorflow.keras import layers

inputs = layers.Input(shape = (1,), dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.Bidirectional(layers.LSTM(64, return_sequences = True))(x)
#print(x.shape)
x = layers.Bidirectional(layers.GRU(64))(x)
print(x.shape)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_4 = tf.keras.Model(inputs, outputs, name = "model_4_bidirectional")

(None, 128)


In [64]:
# Get a summary

model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               74496     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 1,354,625
Trainable params: 1,354,625
Non-trainable params: 0
_________________________________________________________________


In [65]:
# Compile model

model_4.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])

In [66]:
model_4_history = model_4.fit(train_sentences, train_labels, epochs = 5, validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                                                                                                                 "model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20210525-121048
Epoch 1/5
215/215 [==============================] - 7s 23ms/step - loss: 0.1753 - accuracy: 0.9500 - val_loss: 1.0292 - val_accuracy: 0.7651
Epoch 2/5
215/215 [==============================] - 4s 16ms/step - loss: 0.0407 - accuracy: 0.9835 - val_loss: 1.0997 - val_accuracy: 0.7480
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0410 - accuracy: 0.9818 - val_loss: 1.2074 - val_accuracy: 0.7559
Epoch 4/5
215/215 [==============================] - 4s 16ms/step - loss: 0.0364 - accuracy: 0.9826 - val_loss: 1.2858 - val_accuracy: 0.7585
Epoch 5/5
215/215 [==============================] - 4s 16ms/step - loss: 0.0323 - accuracy: 0.9834 - val_loss: 1.6392 - val_accuracy: 0.7520


In [67]:
# Make predictions with out bidiractional ,odel
 
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[5.9797549e-01],
       [8.7858331e-01],
       [9.9993670e-01],
       [1.1313723e-01],
       [9.1661395e-06],
       [9.9987185e-01],
       [9.9927801e-01],
       [9.9999547e-01],
       [9.9998069e-01],
       [9.9968708e-01]], dtype=float32)

In [68]:
# Convert

model_4_pred = tf.squeeze(tf.round(model_4_pred_probs))
model_4_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [69]:
# Calculate the results of our bidiractional model

model_4_results = calculate_results(y_true = val_labels, y_pred = model_4_pred )
model_4_results

{'accuracy': 75.19685039370079,
 'f1': 0.7516029615774639,
 'precision': 0.7515527438106716,
 'recall': 0.7519685039370079}

### Model 5: Conv1D 

We've used CNNs for images but images are typically 2D (hegith x width)...however, our text data is 1d.

The typical structure of a Conv1D model for sequen ces (in our case, text):

```
Inputs(test -> Tokenization -> Embedding -> Cpmv1D -> Pool -> Outputs (class probs)
```

In [70]:
from tensorflow.keras import layers

input = layers.Input(shape = (1,), dtype = "string")
x = text_vectorizer(input)
x = embedding(x)
x = layers.Conv1D(32, 6, activation = 'relu')(x) #kernal size is reffered to as n-gram side (strides rn is 1, but can change to hop) #padding = 'valid' (some words mised) output is smaller 'same' = smae size
x = layers.GlobalMaxPooling1D()(x)
output = layers.Dense(1, activation = "sigmoid")(x)

model_5 = tf.keras.Model(input, output, name = "model_5_conv")


In [71]:
model_5.summary()

Model: "model_5_conv"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
conv1d (Conv1D)              (None, 10, 32)            24608     
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 1,304,641
Trainable params: 1,304,641
Non-trainable params: 0
____________________________________________

In [72]:
# Compiling the model

model_5.compile(loss = 'binary_crossentropy', optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])



In [73]:
# Fitting the model

model_5_history = model_5.fit(train_sentences, train_labels, epochs = 5, 
                              validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(SAVE_DIR, "model_5_conv")])

Saving TensorBoard log files to: model_logs/model_5_conv/20210525-121111
Epoch 1/10
215/215 [==============================] - 5s 18ms/step - loss: 0.1571 - accuracy: 0.9514 - val_loss: 1.0264 - val_accuracy: 0.7546
Epoch 2/10
215/215 [==============================] - 3s 14ms/step - loss: 0.0616 - accuracy: 0.9764 - val_loss: 1.1942 - val_accuracy: 0.7520
Epoch 3/10
215/215 [==============================] - 3s 14ms/step - loss: 0.0496 - accuracy: 0.9786 - val_loss: 1.2682 - val_accuracy: 0.7546
Epoch 4/10
215/215 [==============================] - 3s 14ms/step - loss: 0.0463 - accuracy: 0.9800 - val_loss: 1.3011 - val_accuracy: 0.7507
Epoch 5/10
215/215 [==============================] - 3s 14ms/step - loss: 0.0416 - accuracy: 0.9813 - val_loss: 1.3751 - val_accuracy: 0.7585
Epoch 6/10
215/215 [==============================] - 3s 14ms/step - loss: 0.0386 - accuracy: 0.9824 - val_loss: 1.3395 - val_accuracy: 0.7480
Epoch 7/10
215/215 [==============================] - 3s 14ms/step - 

In [74]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters = 64, kernel_size = 5, activation = "relu")(x)
x = layers.GlobalMaxPool1D()(x)
#x = layers.Dense(64, activation = "relu")(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_6 = tf.keras.Model(inputs, outputs, name = "model_6_Conv1D")

model_6.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])

model_6.summary()

Model: "model_6_Conv1D"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 11, 64)            41024     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 1,321,089
Trainable params: 1,321,089
Non-trainable params: 0
__________________________________________

In [75]:
# Fit the model

model_6_history = model_6.fit(train_sentences, train_labels, epochs = 5,
                              validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(SAVE_DIR, "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20210525-121144
Epoch 1/5
215/215 [==============================] - 5s 19ms/step - loss: 0.1396 - accuracy: 0.9557 - val_loss: 1.0944 - val_accuracy: 0.7493
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0382 - accuracy: 0.9869 - val_loss: 1.2936 - val_accuracy: 0.7559
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0382 - accuracy: 0.9828 - val_loss: 1.3740 - val_accuracy: 0.7507
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0385 - accuracy: 0.9837 - val_loss: 1.3650 - val_accuracy: 0.7493
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0320 - accuracy: 0.9863 - val_loss: 1.3670 - val_accuracy: 0.7507


In [76]:
# Make some prediction with our Conv1D model

model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred = tf.squeeze(tf.round(model_6_pred_probs))
model_6_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [77]:
model_6_results = calculate_results(y_true = val_labels, y_pred = model_6_pred)
model_6_results

{'accuracy': 75.06561679790026,
 'f1': 0.7491705324008289,
 'precision': 0.7507272180149456,
 'recall': 0.7506561679790026}

## Model6: Tensorflow Hub Pretrained Sentence Encoder

Using transfer learning for NLP

In [78]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed([sample_sentence, "When you call the use on a sentence, it utrns it into numbers"])

print(embed_samples[0][:50])

tf.Tensor(
[-0.01157027  0.02485911  0.02878048 -0.012715    0.03971538  0.08827761
  0.02680985  0.05589839 -0.01068729 -0.00597292  0.00639323 -0.0181952
  0.00030814  0.09105889  0.05874645 -0.03180628  0.01512474 -0.05162929
  0.00991367 -0.06865346 -0.04209306  0.02678981  0.03011008  0.00321069
 -0.00337971 -0.04787357  0.02266719 -0.00985925 -0.04063613 -0.01292093
 -0.04666385  0.056303   -0.03949255  0.00517688  0.02495828 -0.07014441
  0.02871509  0.04947684 -0.00633978 -0.08960193  0.02807117 -0.00808362
 -0.01360601  0.0599865  -0.10361787 -0.05195374  0.00232955 -0.0233253
 -0.03758106  0.0332773 ], shape=(50,), dtype=float32)


In [79]:
# transforms THE WHOLE sentence into a 512 dimensional vector

In [80]:
# Create Keras Layer using the USE pretrained layer from tensorflow hub

sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape = [],
                                        dtype = tf.string,
                                        trainable = False,
                                        name = "USE")

In [81]:
# Create model using the sequenctial API

model_7 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation = "relu"),
    layers.Dense(1, activation = 'sigmoid', name = 'output_layer')
], name = "model_7_USE")

# Compile

model_7.compile(loss = 'binary_crossentropy', optimizer = 
                tf.keras.optimizers.Adam(), metrics = ['accuracy'])

model_7.summary()

Model: "model_7_USE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE (KerasLayer)             (None, 512)               256797824 
_________________________________________________________________
dense_6 (Dense)              (None, 64)                32832     
_________________________________________________________________
output_layer (Dense)         (None, 1)                 65        
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [82]:
# Train a classifier on top of USE pretrained embeddings

model_7_history = model_7.fit(train_sentences, train_labels, epochs = 15,
                              validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(SAVE_DIR, "model_7_use")])

Saving TensorBoard log files to: model_logs/model_7_use/20210525-121224
Epoch 1/15
215/215 [==============================] - 7s 23ms/step - loss: 0.5830 - accuracy: 0.7250 - val_loss: 0.4517 - val_accuracy: 0.7953
Epoch 2/15
215/215 [==============================] - 2s 10ms/step - loss: 0.4127 - accuracy: 0.8162 - val_loss: 0.4464 - val_accuracy: 0.8071
Epoch 3/15
215/215 [==============================] - 2s 10ms/step - loss: 0.4027 - accuracy: 0.8235 - val_loss: 0.4346 - val_accuracy: 0.8123
Epoch 4/15
215/215 [==============================] - 2s 10ms/step - loss: 0.3897 - accuracy: 0.8238 - val_loss: 0.4307 - val_accuracy: 0.8136
Epoch 5/15
215/215 [==============================] - 2s 10ms/step - loss: 0.3909 - accuracy: 0.8343 - val_loss: 0.4283 - val_accuracy: 0.8150
Epoch 6/15
215/215 [==============================] - 2s 10ms/step - loss: 0.3763 - accuracy: 0.8349 - val_loss: 0.4335 - val_accuracy: 0.8202
Epoch 7/15
215/215 [==============================] - 2s 9ms/step - lo

In [84]:
model_7_pred_probs = model_7.predict(val_sentences)
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_results = calculate_results(val_labels, model_7_preds)


## Model8 :TF Hub Pretrained USE but with 10% of training data

Transfer learning really helps when you don't have a alarge dataset.

To see how our model performs on a smaller dataset, let's replicate 'model_& except we'll train it on 10% of the data.

In [85]:
# Create subset of 10% of the training data

# NOte

#train_10_percent = train_df_shuffled[["text", "target"]].sample(frac = 0.1, random_state = 42)

#train_sentences_10_percent = train_10_percent["text"].to_list()
#train_labels_10_percent = train_10_percent['target'].to_list()

 **Note** Be *very* carefule when creating training/va;/test splits that you don't leak data across the datasets, otherwise your model evaluation metrics will be wrong. If something looks to good to be true (a model trainied on 10 percent of data doutperforming a mdel trained on100% of the data) trust your gut and o back thorugh to find where the error may lie.

In [86]:
train_10_percent_split = int(0.1* len(train_sentences))
train_10_percent = train_sentences[:train_10_percent_split]
train_labels_10_percent =  train_labels[:train_10_percent_split]

In [87]:
pd.Series(np.array(train_labels_10_percent)).value_counts()

0    406
1    279
dtype: int64

In [88]:
# Check the number of target in our subset of data (ensure ratios are relatively close)

train_df_shuffled["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [89]:
train_df_shuffled['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

To recreeate amodel the same as a previous model you've created, you can use the 'tf.keras.models.clone_model ' method

In [90]:
# Let's build a nodel the same as model_6

model_8 = tf.keras.models.clone_model(model_7)

model_8.compile(loss = 'binary_crossentropy', optimizer= tf.keras.optimizers.Adam(), metrics = ['accuracy'])

model_8.summary()

Model: "model_7_USE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE (KerasLayer)             (None, 512)               256797824 
_________________________________________________________________
dense_6 (Dense)              (None, 64)                32832     
_________________________________________________________________
output_layer (Dense)         (None, 1)                 65        
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [91]:
# Fit the model on 10% training data

model_8_history = model_8.fit(train_10_percent, train_labels_10_percent, epochs = 5,
                              validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(SAVE_DIR, 'model_8_correct_split')])

Saving TensorBoard log files to: model_logs/model_8_correct_split/20210525-121403
Epoch 1/5
22/22 [==============================] - 4s 113ms/step - loss: 0.6781 - accuracy: 0.6558 - val_loss: 0.6385 - val_accuracy: 0.7520
Epoch 2/5
22/22 [==============================] - 0s 18ms/step - loss: 0.5938 - accuracy: 0.8415 - val_loss: 0.5781 - val_accuracy: 0.7651
Epoch 3/5
22/22 [==============================] - 0s 18ms/step - loss: 0.5171 - accuracy: 0.8395 - val_loss: 0.5327 - val_accuracy: 0.7664
Epoch 4/5
22/22 [==============================] - 0s 20ms/step - loss: 0.4610 - accuracy: 0.8354 - val_loss: 0.5043 - val_accuracy: 0.7677
Epoch 5/5
22/22 [==============================] - 0s 18ms/step - loss: 0.4046 - accuracy: 0.8630 - val_loss: 0.4930 - val_accuracy: 0.7664


In [93]:
model_8_pred_probs = model_8.predict(val_sentences)

model_8_preds = tf.squeeze(tf.round(model_8_pred_probs))

model_8_results = calculate_results(val_labels, model_8_preds)

#print(results)

## Comparing the performance of each of our models

In [99]:
# Combine model results into a Datafrmae 

all_model_results = pd.DataFrame({"0_baseline": baseline_results, "1_simple_dense": model_1_results,
                                  "2_lstm": model_2_results, "3_gru": model_3_results,
                                  "4_bidirectional": model_4_results, "5_conv1d": model_6_results, 
                                  "6_tf_hub_use_encoder": model_7_results, 
                                  "7_tf_hub_use_encoder_10_percent":model_8_results})
all_model_results

,0_baseline,1_simple_dense,2_lstm,3_gru,4_bidirectional,5_conv1d,6_tf_hub_use_encoder,7_tf_hub_use_encoder_10_percent
accuracy,79.265092,76.509186,76.640420,76.377953,75.196850,75.065617,82.808399,76.640420
precision,0.811139,0.766217,0.766590,0.763471,0.751553,0.750727,0.830367,0.767452
recall,0.792651,0.765092,0.766404,0.763780,0.751969,0.750656,0.828084,0.766404
f1,0.786219,0.763156,0.765121,0.762981,0.751603,0.749171,0.826759,0.764542


## Uploading model's training logs to TensorBoard.dev

In [100]:
!tensorboard dev upload --logdir ./model_logs/ \
 --name "NLP Modelling Experiments" \
 --description "comparing model architectures" \
 --one_shot # exit the uploader once finished

2021-05-25 12:23:14.181157: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Data for the "text" plugin is now uploaded to TensorBoard.dev! Note that uploaded data is public. If you do not want to upload data for this plugin, use the "--plugins" command line argument.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/sy6KOA6sR02cP3Y06urW1w/

[2021-05-25T12:23:15] Started scanning logdir.
[2021-05-25T12:23:23] Total uploaded: 280 scalars, 0 tensors, 8 binary objects (4.7 MB)
[2021-05-25T12:23:23] Done scanning logdir.


Done. View your TensorBoard at https://tensorboard.dev/experiment/sy6KOA6sR02cP3Y06urW1w/


Now I've ran the cell above, my modelling experiments are visable on TensorBoard.dev:

In [101]:
# If you need to delete an experiment from TensorBoard, you can run the following:

# See the previous Tensorboard Dev experiments you've run

!tensorboard dev list

2021-05-25 12:26:57.314323: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Data for the "text" plugin is now uploaded to TensorBoard.dev! Note that uploaded data is public. If you do not want to upload data for this plugin, use the "--plugins" command line argument.
https://tensorboard.dev/experiment/sy6KOA6sR02cP3Y06urW1w/
	Name                 NLP Modelling Experiments
	Description          comparing model architectures
	Id                   sy6KOA6sR02cP3Y06urW1w
	Created              2021-05-25 12:23:15 (3 minutes ago)
	Updated              2021-05-25 12:23:23 (3 minutes ago)
	Runs                 16
	Tags                 3
	Scalars              280
	Tensor bytes         0
	Binary object bytes  4927818
https://tensorboard.dev/experiment/8t6OOlNlQGuDRfPjDPPvew/
	Name                 NLP Modelling Experiments
	Description          comparing model architectures
	Id                   8t6OOlNlQGuDRfPjDPPvew
	Created 